<a href="https://colab.research.google.com/github/tkota0726/Jupyter/blob/master/Weather_blog_ScikitLearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 入力値
  - 4日前のPV
  - 3日前のPV
  - 2日前のPV
  - 1日前のPV
  
  Data Description
  "# Renewables.ninja PV (hourly data, 1985-2016) - ninja_pv_country_EE_merra-2_corrected - Version: 1.1 - License: https://creativecommons.org/licenses/by-nc/4.0/ - Reference: https://doi.org/10.1016/j.energy.2016.08.060",
"# Units: time in UTC, other columns are capacity factors [0-1].  Bias corrected using national generation data.",
  
  "# Renewables.ninja Weather (hourly data, 1980-2016) - Estonia - ninja_weather_country_EE_merra-2_population_weighted.csv - Version: 1.0 - License: https://science.nasa.gov/earth-science/earth-science-data/data-information-policy - Reference: https://doi.org/10.1175/JCLI-D-16-0758.1",,,,,,,,
"# Units: time in UTC, other columns are average weather variables, population weighted. precipitation in mm/hour (PRECTOTLAND in MERRA-2). temperature at 2 metres above ground in degrees C (T2M in MERRA-2). irradiance at ground level (_surface) and top of atmosphere (_toa) in W/m^2 (SWGDN and SWTDN in MERRA-2). snowfall in mm/hour (PRECSNOLAND in MERRA-2). snow_depth in cm (SNODP in MERRA-2). cloud_cover as a fraction [0,1] (CLDTOT in MERRA-2). air_density at ground level in kg/m^3 (RHOA in MERRA-2).",,,,,,,,

In [0]:
from math import sqrt
from numpy import concatenate
import matplotlib.pyplot as plt
import pandas as pd
from pandas import Series
from datetime import datetime
import numpy as np
import time
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
from google.colab import files
import os.path

!pip install pydrive
import pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

%matplotlib inline
drive.mount('/content/gdrive')

In [0]:
def dataDriveExtract(id, file):
  
  auth.authenticate_user()#Auth check
  gauth = GoogleAuth() #google Auth Instance
  gauth.credentials = GoogleCredentials.get_application_default() # credential part
  drive = GoogleDrive(gauth)# put the credential information to drive

  downloaded = drive.CreateFile({'id':str(id)})# Specify the file location 
  downloaded.GetContentFile(file)# download the file with the file name
  data = pd.read_csv(file, encoding = 'utf-8', index_col='time')  # load the data via pandas
  data.index = pd.to_datetime(data.index) #Date Index formatting
  return data


con = "weather"
if con == "weather":
  data_weather = dataDriveExtract('1Y4-cvQC5aALc08HW5Q_7bnFphEONO1aW', 'EE-Weather-1985-2016.csv')
  data_2016weather = data_weather["2016-01-01":"2016-12-31"]
  data_2016weather.info() # show data info
#   print(data_2016weather.head()) # show data info
#   print(data_2016weather.head(1))
elif con == "pv":
  data_pv = dataDriveExtract()
  data_2016pv = data_weather["2016-01-01":"2016-12-31"]
  data.info() # show data info
  


In [0]:
data_temp = data_2016weather.loc[:, 'temperature']
data_temp.plot()
data_temp.describe()

In [0]:
#データ数をcount_sに代入
count_s = len(data_temp)

# 上昇率を格納するリストを作成
inc_rate =[]
#気温の上昇率を計算
for i in range(1, count_s):
  try:
    #上昇率を計算する式
    inc_rate.append(float(data_temp[i] - data_temp[i-1]) / float(data_temp[i-1]) )
  except ZeroDivisionError:
     #ゼロが分母にきた場合は、０に変換
    inc_rate.append(0)

#　気温上昇率のデータ数を格納
count_m = len(inc_rate)
#　気温上昇率のデータ数を表示
print(count_m)

In [0]:
from sklearn import svm

#前日までの4連続の上昇率データ
successive_data = []

# 正解値　価格上昇: 1 価格低下: 0
answers = []

lb = 7 # lookback number
for i in range(lb, count_m):

  successive_data.append([inc_rate[i-7], inc_rate[i-6], inc_rate[i-5], inc_rate[i-4], inc_rate[i-3], inc_rate[i-2], inc_rate[i-1]])

  if inc_rate[i] > 0:
    answers.append(1)
  else:
    answers.append(0)
    
# print(successive_data)
# print(answers)

In [0]:
lb = 7
k = 10
l_2d_ok = [([i-k]) * lb for i in range(3+1)]
print(l_2d_ok)

In [0]:
#データ数
n = len(successive_data)
print(n)

m = len(answers)
print(m)

In [0]:
#線形サポートベクタマシーン
clf = svm.LinearSVC()

tr= 8500
# Training
clf.fit(successive_data[:tr], answers[:tr])

te = n - tr
# Test
expected = answers[te:]
# predict
predicted = clf.predict(successive_data[-te:])

# 正解率計算
correct = 0
wrong = 0
for i in range(te):
  if expected[i] == predicted[i]:
    correct += 1
  else:
    wrong += 1
      
print("正解率:  " + str(correct / (correct+wrong) * 100)       + "%")

In [0]:
plt.figure(figsize=(20,8))
plt.plot(expected[0:250])
plt.plot(predicted[0:250])